In [1]:
%matplotlib inline

import numpy as np

import time
import math
from numpy.linalg import inv
import matplotlib
from scipy.stats import binom
from scipy.stats import gamma
from scipy.stats import laplace
from scipy.optimize import minimize
from scipy import integrate
from scipy.optimize import minimize
from scipy.misc import derivative
from matplotlib import pyplot as plt
from scipy.stats import multivariate_normal
from scipy.special import gamma 


 <h2> Mean Field Approximation Example: </h2>

Example 1: The Univariate Gaussian (Example taken from Bishop´s Pattern Recognition Book)

 Assume we have unidimensional data drawn from a unknown Normal distribution. Suppose we want to estimate the parameters
 $\mu$ and $\sigma$ using a Bayesian approach. Then we have:

 \begin{eqnarray}
   P(\mu,\sigma|D) &=& P(D|\mu,\sigma) P(\mu,\sigma)\\
                   &=& P(D|\mu,\sigma) P(\mu|\sigma)P(\sigma)\\
 \end{eqnarray}
 
  Given that $P(D|\mu,\sigma)$ is Gaussian, we use conjugate priors for $\mu$ and $\sigma$:

 $P(\mu|\sigma) = \mathcal{N}(\mu|\mu_0,(\lambda_0 \sigma)^{-1})$
 
 $P(\sigma) = Gamma(\sigma|a_0,b_0) = \frac{{b_0}^{a_0}}{\Gamma(a_0)} \sigma^{a_0-1} e^{-b_0 \sigma}$

  Together these distributions constitute a Gaussian-Gamma posterior for $P(\mu,\sigma|D)$:
  
  $P(\mu,\sigma|m,\lambda,\alpha,\beta) = \frac{\beta^\alpha \sqrt{\lambda}}{\Gamma(\alpha)\sqrt{2 \pi}} \sigma^{\alpha - \frac{1}{2}} e^{-\beta \sigma} e^{-\frac{\lambda \sigma (\mu-m)}{2}}$


 In this case the posterior can be found exactly and is a Gaussian-Gamma distribution, but for tutorial purposes lets approximate it using variational inference.
 
 We will approximate the posterior distribution $q(\mu,\sigma)$ as the product of independent distributions:
 
   $q(\mu,\sigma) = q_{\mu}(\mu)q_{\sigma}(\sigma)$
   
 From the mean field equations we know that:
 
 \begin{eqnarray}
 \ln q_{\mu}(\mu) &=& \mathbb{E}_{q_{\sigma}}[\ln p(D|\mu,\sigma) + \ln p(\mu|\sigma)] + const\\
                     &=& - \frac{\mathbb{E}_{q_{\sigma}}[\sigma]}{2} \Big\{ \lambda_0 (\mu - \mu_0)^2 + \sum_{n=1}^{N} (x_n - \mu)^2 \Big\} + const 
 \end{eqnarray}
 
 Note that the prior $p(\sigma)$ is constant for $q_{\mu}(\mu)$. Completing the square over $\mu$ we can see that $q_{\mu}(\mu)$ is a Gaussian $\mathcal{N}(\mu | \mu_N, \lambda_N^{-1} )$
 where:
 
 \begin{eqnarray}
    \mu_N &=& \frac{\lambda_0 \mu_0 + N \overline{x}}{\lambda_0 + N}\\
    \lambda_N &=& (\lambda_0 + N) \mathbb{E}_{q_{\sigma}}[\sigma] 
 \end{eqnarray}

 Similarly, the optimal solution for $q_{\sigma}(\sigma)$ is:
 
 \begin{eqnarray}
  \ln q_{\sigma}(\sigma) &=& \mathbb{E}_{q_{\mu}}[\ln p(D |\mu, \sigma) + \ln p(\mu|\sigma) + \ln p(\sigma)] + const\\
                           &=& (a_0 - 1) \ln\sigma - b_0\sigma + \frac{N}{2} \ln\sigma\\
                     && - \frac{\sigma}{2} \mathbb{E}_{q_{\mu}} \Bigg[ \sum_{n=1}^{N} (x_n - \mu)^2 + \lambda_0 (\mu - \mu_0)^2 \Bigg] + const
 \end{eqnarray}
 
 Hence $q_{\sigma}(\sigma)$ is a Gamma distribution $Gamma(\sigma | a_N,b_N)$ with parameters:
 
 \begin{eqnarray}
    a_N &=& a_0 + \frac{N}{2} \\
    b_N &=& b_0 + \frac12 \mathbb{E}_{q_{\mu}} \Bigg[ \sum_{n=1}^{N} (x_n - \mu)^2 + \lambda_0 (\mu - \mu_0)^2 \Bigg]
 \end{eqnarray}
 
 Using the known value for the expectation of $\sigma$,we have that $\mathbb{E}_{q_{\sigma}}[\sigma] = \frac{a_N}{b_N}$.
Also to simplify the expressions we can assume non informative priors:

$\lambda_0 = a_0 = b_0 = \mu_0 = 0$


\begin{eqnarray}
    \frac{1}{\mathbb{E}_{q_{\sigma}}[\sigma]} &=& \mathbb{E}_{q_{\mu}} \Bigg [ \frac{1}{N} \sum_{n=1}^{N}(x_n - \mu)^2  \Bigg]
                                 &=& \overline{x^2} - 2\overline{x}\mathbb{E}_{q_{\mu}}[\mu] + \mathbb{E}_{q_{\mu}}[\mu^2]
\end{eqnarray}

Then the first and second moments of $q_{\mu}(\mu)$ are (recall that is a Gaussian):

$\mathbb{E}_{q_{\mu}}[\mu] = \overline{x}$

$\mathbb{E}_{q_{\mu}}[\mu^2] = \overline{x}^2 + \frac{1}{N \mathbb{E}_{q_{\sigma}}[\sigma]}$

 Substituting with the previous equation we have:

$\mathbb{E}_{q_{\sigma}}[\sigma] = \frac{N-1}{\sum_{n=1}^{N}(x_n - \overline{x})^2}$ (Note that this is the inverse of the variance of a univariate Gaussian distribution)



In [2]:
l_0 = 0
a_0 = 0
b_0 = 0
mu_0 = 0

def calculate_parameters(x,E_mu, E_sigma, a_0 = 0, mu_0 = 0, l_0 = 0):
    N = len(x)
    x_mean = np.mean(x)
    mu_N = (l_0*mu_0 + N*x_mean)/(l_0 + N)
    l_N_inverse = 1/((l_0 + N)*E_sigma)
    a_N = a_0 + N/2
    b_N = 1/(E_sigma/a_N)
    return mu_N, l_N_inverse, a_N, b_N
    
def e_q_sigma(x,E_mu,E_sigma):
    N = len(x)
    x_mean = np.mean(x)
    x_square_mean = np.mean(sum(x**2))
    E_mu_square = x_mean**2 + 1/(N*E_sigma)
    return (1/(x_square_mean - 2*x_mean*E_mu + E_mu_square))


In [3]:
%matplotlib qt
mu = 3
sigma = 1
N = 30
x1 = np.random.normal(mu, sigma, N)
iterations = 10

gaussian = lambda x,mu,sigma: (1/(np.sqrt(2 *sigma* np.pi))) *np.exp( - (x - mu)**2 / (2 * sigma)) 
gamma_pdf = lambda s,a,b: ((b**a)/gamma(a))*(s**(a-1))*np.exp(-1*b*s)

E_mu = np.mean(x1)
E_sigma = 1
x, y = np.mgrid[0:6:.1, 0:1:.1]
pos = np.empty(x.shape + (2,))
pos[:, :, 0] = x; pos[:, :, 1] = y

#nx, ny = (3, 2)
#x = np.linspace(0, 1, nx)
#y = np.linspace(0, 1, ny)
#xv, yv = meshgrid(x, y)


for i in range(iterations):
    E_sigma = e_q_sigma(x1,E_mu,E_sigma)
    mu_N, l_N_inverse, a_N, b_N = calculate_parameters(x1,E_mu,E_sigma)
    #print(mu_N)
    #print(l_N_inverse)
    #print(a_N)
    #print(b_N)
    
q_mu_sigma = lambda x_,s_: (gaussian(x_,mu_N,l_N_inverse))*(gamma_pdf(s_,a_N,b_N)) 
integral = sum(sum(q_mu_sigma(x,y)))
print(integral)
plt.contour(x, y, (q_mu_sigma(x,y)))
plt.show()

    

5.06079672765e-149


Parámetro de gaussian gamma
lambda = N
ALPHA = N/2
beta = 1/2(sum(x_i**2))
m = (1/N)*sum(x_i)
P(μ,σ|m,λ,α,β)=βαλ√Γ(α)2π√σα−12e−βσe−λσ(μ−m)2

In [4]:
x, y = np.mgrid[0:6:.1, 0:1:.1]
pos = np.empty(x.shape + (2,))
pos[:, :, 0] = x; pos[:, :, 1] = y

l1 = len(x1)
a1 = l1/2
b1= (1/2)*sum(x1**2)
m1 = (sum(x1))/l1
gaussian_gamma = lambda x_,y_, l, a, b, m : (((b**a)*np.sqrt(l)) / (gamma(a)*np.sqrt((np.pi)*2)))* (y_**(a-(1/2))) * np.exp(-1*b*y_) * np.exp(-(l*y_*(x_-m)**2)/2)
gaussian_gamma_plt = lambda x_,y_: gaussian_gamma(x_,y_,l1,a1,b1,m1)
#integral2 = sum(sum(gaussian_gamma_plt(x,y))) 
plt.contour(x, y, gaussian_gamma_plt(x,y))
plt.show()